<a href="https://colab.research.google.com/github/mistryishan25/Comprehensive-Project-SEM8/blob/master/notebooks/SVO_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Installing relevant dependencies
!pip install langid
!pip install google_trans_new
!python -m spacy download en_core_web_md
!pip install contractions

In [ ]:
# Install spacy web-core-md

In [ ]:
# Import requisite libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
import string
import bs4
from bs4 import BeautifulSoup as bs
import unicodedata
import pickle
import contractions

# Fast NLP
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

# Language Detection. 
import langid

## N-Gram based approach
from nltk.classify.textcat import TextCat

# Google Translator
from google_trans_new import google_translator

import warnings
warnings.filterwarnings('ignore')

from collections import Counter, defaultdict
import time
from datetime import datetime
import re
import pickle
from IPython.display import display, HTML, display_html

#Define the maximum and minimum no. of rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
# Downloading the relevant libraries and dependencies in NLTK module for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')                                                                     
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Importing data
path = "/content/drive/MyDrive/CP Sem-8/Dataset(s)/Youtube/Fun/Jimmy Kemme-Trump.csv"
df = pd.read_csv(path)

In [ ]:
# Look at data
display(df.head())
print()

# Look at the shape of data
display(df.shape)
print()

# Look at data types for potential mismatch 
display(df.iloc[:5, :5].dtypes)
print()

# Data info
display(df.iloc[:5, :5].info())
print()

# Check for null values
display(df.iloc[:5, :5].isnull().any())
print()

# Check for descriptive statistics 
display(df.describe(percentiles = [0.25, 0.50, 0.75, 0.99]))

,Name,Comment,Time,Likes,Reply Count,Reply Author,Reply,Published,Updated
0,River Amazon,woe si great kid,2022-03-26T14:28:20Z,0.0,0.0,NaN,NaN,NaN,NaN
1,DIANA KLEINHAMPL,This kid is a comic genius.,2022-03-26T07:59:46Z,0.0,0.0,NaN,NaN,NaN,NaN
2,Pro Gaming,"Ellen, el alofoke de usa",2022-03-25T15:20:02Z,0.0,0.0,NaN,NaN,NaN,NaN
3,Super Cyber,superkid....,2022-03-25T13:05:50Z,0.0,0.0,NaN,NaN,NaN,NaN
4,Aleksandar,That kid is amazing.. and the bond with grandp...,2022-03-25T07:44:17Z,0.0,0.0,NaN,NaN,NaN,NaN


(9305, 9)

Name            object
Comment         object
Time            object
Likes          float64
Reply Count    float64
dtype: object


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         5 non-null      object 
 1   Comment      5 non-null      object 
 2   Time         5 non-null      object 
 3   Likes        5 non-null      float64
 4   Reply Count  5 non-null      float64
dtypes: float64(2), object(3)
memory usage: 328.0+ bytes


None

Name           False
Comment        False
Time           False
Likes          False
Reply Count    False
dtype: bool

,Likes,Reply Count
count,7563.000000,7563.000000
mean,54.066508,0.230332
std,598.343152,3.107242
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
99%,1227.660000,6.000000
max,20454.000000,202.000000


In [ ]:
# 'Date-time object' is represented as 'object' data type. Also change the data-type of 'no. of likes' and 'reply count' to 'int64'
df.rename(columns = {'Time': 'TimeCombined'}, inplace = True)
df['TimeCombined'] = pd.to_datetime(df['TimeCombined'], infer_datetime_format=True)
df['Date'] = df['TimeCombined']
df['Time'] = df['TimeCombined']
df['TimeCombined'] = df['TimeCombined'].astype(str).str.split()

#Basic preprocessing for time
for index, _ in df.iterrows():
  if len(df.loc[index, 'TimeCombined']) > 1:
    df.loc[index, 'Date'] = df.loc[index, 'TimeCombined'][0]
    df.loc[index, 'Time'] = df.loc[index, 'TimeCombined'][1]
    
  else:
    df.loc[index, 'Date'] = 0
    df.loc[index, 'Time'] = 0

# Drop the time-combined column later

In [ ]:
# Initialize the stemmer and lemmatizer
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer()
translator = google_translator()

In [ ]:
adv_tag_list = ['RB', 'RBR', 'RBS']
adj_tag_list = ['JJ', 'JJR', 'JJS']
verb_tag_list = ['VB', 'VBD', 'VBG', 'VBN', 'VBP']

def cleanComment(row, col):
  modified_desc_text = []
  textNew = ""

  try:
    text = row[col]
    text = str(text).strip()
    text = bs(text, 'html.parser').get_text()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub("http\S*\s", "", text)
    text = re.sub("(?:[a-zA-Z]\.){2,}", " ", text)
    
    # for word in process_title1.split():
    # word_tag_tuple = nltk.str2tuple(word)
    
    expanded_words = []
    for word in text.split():
        
        # using contractions.fix top expand the shortened words
        expanded_words.append(contractions.fix(word))
        
    text = ' '.join(expanded_words)

    words = nltk.word_tokenize(text)

    words = [lemmatizer.lemmatize(word) for word in words] 
    
    tagged_desc_list = nltk.pos_tag(words)

    for index, (word, tag) in enumerate(tagged_desc_list):

      if tag != 'NNP':
        word = word.lower()

      modified_desc_text.append(word)

    textNew = ' '.join(modified_desc_text)

  except Exception as e:
    print(str(e))

  return nlp(textNew)

In [ ]:
df['Comment_New'] = df.apply(lambda row: cleanComment(row, 'Comment'), axis = 1)
df['Replies_New'] = df.apply(lambda row: cleanComment(row, 'Reply'), axis = 1)

In [ ]:
# First a# text = '''hello @NITHISH RAGHAV and @Krish Naik sir currently i am in a third year of B.tech in computer science and i  also want to work in a  data science domain but i don't know where to start learning for ML ,so i started learning from your ML playlist from your website but i have one issue as i have only 52 percentage in my 12th standard and some big companies have a criteria that if you have percentage greater than 60 then only you are eligible for sitting in placement or for the interview so what can be done now as due to that criteria i am not able to attend many company interviews so can what could be done further sir i am confused can you pls help me'''
# text = cleanComment(text)
# Analyze the language used in comments. First approach is using 'Langid' tool
# langs = df['Comment'].apply(langid.classify).to_frame()

# Analyse the labels
# lang_labels = langs['Comment_New'].apply(lambda row : row[0])
# lang_labels.value_counts()
# text = '''hello @NITHISH RAGHAV and @Krish Naik sir currently i am in a third year of B.tech in computer science and i  also want to work in a  data science domain but i don't know where to start learning for ML ,so i started learning from your ML playlist from your website but i have one issue as i have only 52 percentage in my 12th standard and some big companies have a criteria that if you have percentage greater than 60 then only you are eligible for sitting in placement or for the interview so what can be done now as due to that criteria i am not able to attend many company interviews so can what could be done further sir i am confused can you pls help me'''

# Let's analyse the first ten sentences of our dataset and their corresponding languages
# df['Comment_New'].head(40)

# Drop the rows with Non-English sentences
# indices = []
# for index, row in langs.iterrows():
#   if row['Comment_New'][0] != 'en':
#     indices.append(index)

# Dropping rows with Non-English labels
# df.drop(df.index[indices], axis = 0, inplace = True)

# Next step involves creating SVO tuples
# Merge all the comments together

# merged_text = ""
# for _, row in df.iterrows():
#   text = str(row['Comment_New'])
#   text = text.strip()
#   merged_text += text + '.'

In [ ]:
# Lets' review a section of the merged text with POS tagging and relevant dependencies
# span = doc[0:10]
# displacy.render(span, style='dep', jupyter=True)

In [ ]:
# Defining the subject, verb and predicates
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
VERBS = ['ROOT', 'advcl']
OBJECTS = ["dobj", "dative", "attr", "oprd", 'pobj']
ENTITY_LABELS = ['PERSON', 'NORP', 'GPE', 'ORG', 'FAC', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART']

In [ ]:
# Helper function for creating SVO triples
# The code has been borrowed from   
def create_svo_lists(text):
  subject_ls = []
  verb_ls = []
  object_ls = []


  for token in text:
    if token.dep_ in SUBJECTS:
      subject_ls.append((token.lower_, token.idx))

    elif token.dep_ in VERBS:
      verb_ls.append((token.lemma_, token.idx))

    elif token.dep_ in OBJECTS:
      object_ls.append((token.lower_, token.idx))
  
  return subject_ls, verb_ls, object_ls


def create_svo_triples(row, col):
  subject_ls, verb_ls, object_ls = create_svo_lists(row[col])
  graph_tup_ls = []
  dedup_tup_ls = []
  clean_tup_ls = []
    
  for subj in subject_ls:
    for obj in object_ls:

      dist_ls = []

      for v in verb_ls:
            #Assemble a list of distances between each object and each ver
            dist_ls.append(abs(obj[1] - v[1]))

      # Get the index of the verb with the smallest distance to the object 
      # and return that verb
      index_min = min(range(len(dist_ls)), key=dist_ls.__getitem__)
      graph_tup_ls.append([subj[0], verb_ls[index_min][0], obj[0]])

  return graph_tup_ls

In [ ]:
# Creating triples
df['SVO_Comment'] = df.apply(lambda row: create_svo_triples(row, 'Comment_New'), axis = 1)
df['SVO_Reply'] = df.apply(lambda row: create_svo_triples(row, 'Replies_New'), axis = 1)

In [ ]:
# Save as csv
df.to_csv('/content/drive/MyDrive/CP Sem-8/Dataset(s)/Processed Data/Youtube/Jimmy-Kemmel.csv')